In [ ]:
efficientnet, preprocess_input = Classifiers.get('resnet50')
model = efficientnet(
    input_shape=(32, 32, 32, 1),
    weights=None,
    include_top=False  # Exclude the original top layer
)
cut_index = None
for i, layer in enumerate(model.layers):
    if layer.name == 'stage4_unit1_bn1':
        cut_index = i
        break
#model.summary()
# If the layer is found, create a new model up to that layer
if cut_index is not None:
    model = Model(inputs=model.input, outputs=model.layers[cut_index].output)
else:
    print("Layer 'stage4_unit1_relu1' not found in the model.")
# Add a GlobalAveragePooling3D layer
x = GlobalAveragePooling3D()(model.output)

# Add the output layer with softmax activation for classification
num_classes = 1  # Adjust based on your classification task
output_tensor = Dense(num_classes, activation='sigmoid')(x)

# Create the new model with the added output layer
model = Model(inputs=model.input, outputs=output_tensor)
model.load_weights('../weights_resnet50_aneu_xxl_ep23.h5')# Stenosis weights_resnet50_steno_xxl_seg_ep25
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0003)
loss_fn = tf.keras.losses.CategoricalCrossentropy()
# Use SparseCategoricalCrossentropy loss function
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
import numpy as np
import nibabel as nib
def reconstruct_3d_shape(data_folder, model, stride, num_images=17):
    reconstructed_shapes = []
    print("Processing data from:", data_folder)
    # Counter to keep track of the number of processed images
    num_processed_images = 0
    
    for filename in os.listdir(data_folder):
        if filename.endswith('.nii.gz'):
            print(filename)
            filepath = os.path.join(data_folder, filename)
            img = nib.load(filepath)
            image_data = img.get_fdata()
            
            # Get the shape of the image
            image_shape = image_data.shape
            
            # Calculate the number of boxes along each dimension
            num_boxes_x = (image_shape[0] - 32) // stride + 1
            num_boxes_y = (image_shape[1] - 32) // stride + 1
            num_boxes_z = (image_shape[2] - 32) // stride + 1
            
            # Initialize the reconstructed shape for the current image
            reconstructed_shape = np.zeros((num_boxes_x, num_boxes_y, num_boxes_z))
    
            # Iterate over the image with the specified stride
            for i in range(0, image_shape[0] - 32 + 1, stride):
                for j in range(0, image_shape[1] - 32 + 1, stride):
                    for k in range(0, image_shape[2] - 32 + 1, stride):
                        # Extract the current box
                        box = image_data[i:i+32, j:j+32, k:k+32]
                        box = np.expand_dims(box, axis=0)
                        box = np.expand_dims(box, axis=-1)
                        # Evaluate the box with the model
                        predictions = model.predict(box, verbose=0)
                        
                        # Store the predictions in the reconstructed shape
                        x_idx = i // stride
                        y_idx = j // stride
                        z_idx = k // stride
                        reconstructed_shape[x_idx, y_idx, z_idx] = predictions
            
            # Append the reconstructed shape to the list
            reconstructed_shapes.append(reconstructed_shape)
            
            # Increment the counter
            num_processed_images += 1
            
            # Break out of the loop if the number of processed images reaches num_images
            if num_processed_images == num_images:
                break  
    return reconstructed_shapes


# Define the folders
steno_train_folder =
healthy_train_folder = 
stride = 4
reconstructed_aneu = reconstruct_3d_shape(steno_train_folder, model, stride)
reconstructed_healthy = reconstruct_3d_shape(healthy_train_folder, model, stride)

In [ ]:
from scipy.ndimage import label
def find_patches(image, threshold, min_patch_size):
    # Create a copy of the image to avoid modifying the original
    image_copy = np.copy(image)

    # Apply threshold to identify low-value voxels
    image_copy[image_copy >= threshold] = 0
    
    # Calculate the total number of non-zero values in the image
    remaining_non_zero_count = np.count_nonzero(image_copy)

    # Get the dimensions of the image
    x_len, y_len, z_len = image_copy.shape

    # Initialize a mask to mark voxels belonging to patches
    patch_mask = np.zeros_like(image_copy, dtype=bool)

    # Define neighbor offsets
    offsets = [(dx, dy, dz) for dx in range(-1, 2) for dy in range(-1, 2) for dz in range(-1, 2) if (dx, dy, dz) != (0, 0, 0)]

    # Iterate over each voxel and check neighbors to find patches
    for x in range(1, x_len - 1):
        for y in range(1, y_len - 1):
            for z in range(1, z_len - 1):
                if image_copy[x, y, z] > 0:
                    # Check if any neighbor is non-zero
                    has_non_zero_neighbor = any(image_copy[x + dx, y + dy, z + dz] > 0 for dx, dy, dz in offsets)
                    if has_non_zero_neighbor:
                        patch_mask[x, y, z] = True

    # Perform connected component analysis to find connected regions of non-zero voxels
    labeled_mask, num_regions = label(patch_mask)

    # Initialize counter for patches
    patch_count = 0

    # Iterate over each region and check if it meets the minimum patch size requirement
    for label_idx in range(1, num_regions + 1):
        region_mask = labeled_mask == label_idx
        region_size = np.sum(region_mask)
        if region_size >= min_patch_size:
            patch_count += 1

    return patch_count, remaining_non_zero_count

In [ ]:
def process_reconstructed_images(reconstructed_images,threshold=0.0000046, min_patch_size=13):
    counts_list = []
    nonzero_counts_list = []
    for image in reconstructed_images:
        counts, nonzero_counts = find_patches(image,threshold, min_patch_size)
        counts_list.append(counts)
        nonzero_counts_list.append(nonzero_counts)
    return counts_list, nonzero_counts_list

def process_lists(lists, threshold):
    total_count = sum(lst > threshold for lst in lists)
    return total_count


aneu_countus,nonzeros_aneu = process_reconstructed_images(reconstructed_aneu)


threshold2 = 0.5


healthy_countus,nonzeros_healthy = process_reconstructed_images(reconstructed_healthy)

aneu_countss = process_lists(aneu_countus, threshold2) #IF 0 then no stenosis or aneurysm found